<a href="https://colab.research.google.com/github/YuHsin-Chen1221/AI-Reasoning-Model-for-Therapeutic-Decision-making/blob/main/Multi_Agent_Framework_Subtasks_Classibication_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain transformers

In [4]:
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.3 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [13]:
# Tasks classification router with LangChain

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

API_KEY = "YOUR_GOOGLE_API_KEY"

task_labels = [
    "Clinical Trial Analysis",
    "Drug Interaction",
    "Professional Diagnosis Suggestion",
    "Treatment Recommendation",
    "Biomedical Literature Understanding"
]

system_prompt = """
You are a medical task classifier that examine the user query and determine a relevant subtask:

Query = {user_input}
Subtasks = [
    "Clinical Trial Analysis",
    "Drug Interaction Analysis",
    "Professional Diagnosis Suggestion",
    "Treatment Recommendation",
    "Biomedical Literature Understanding"
]

Response with only the **task label**
"""


llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key = API_KEY)
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{user_input}")
])

classifier_chain = chat_prompt | llm

"""
# test responses
query = "Which medication is best for lowering LDL cholesterol in elderly patients?"
response = classifier_chain.invoke({"user_input": query})
response.content
"""

'Treatment Recommendation'

In [49]:
!pip install --upgrade huggingface_hub langchain langchain_community
!pip install accelerate

In [65]:
# Multi-agent Define Functions

from huggingface_hub import InferenceClient

HF_TOKEN = "YOUR_HUGGINGFACE_TOKEN"
client = InferenceClient(token=HF_TOKEN)

def handle_clinical_trial(query):
    response = client.text_generation(model="emilyalsentzer/Bio_ClinicalBERT", prompt=query)
    return response.generated_text

def handle_drug_interaction(query):
    response = client.text_generation(model="microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract", prompt=query)
    return response.generated_text

def handle_diagnosis(query):
    response = client.chat_completion(model="mistralai/Mistral-7B-Instruct-v0.3", messages=[{"role":"user","content":query}])
    return response.choices[0].message.content

def handle_treatment(query):
    response = client.chat_completion(model="mistralai/Mistral-7B-Instruct-v0.3", messages=[{"role":"user","content":query}])
    return response.choices[0].message.content

def handle_biomedical_research(query):
    response = client.text_generation(model="microsoft/BioGPT-Large", prompt=query)
    return response.generated_text

def handle_irrelevant(query):
    return "抱歉，我無法處理這個問題，請換個更相關的醫學問題試試。"


In [57]:
# 分流器 based on subtask type

def agent_router(query):
    # 先呼叫分類器，得到任務標籤
    response = classifier_chain.invoke({"user_input": query})
    task_label = response.content.strip()

    # 路由邏輯
    if task_label == "Clinical Trial Analysis":
        return handle_clinical_trial(query)
    elif task_label == "Drug Interaction Analysis":
        return handle_drug_interaction(query)
    elif task_label == "Professional Diagnosis Suggestion":
        return handle_diagnosis(query)
    elif task_label == "Treatment Recommendation":
        return handle_treatment(query)
    elif task_label == "Biomedical Literature Understanding":
        return handle_biomedical_research(query)
    else:
        return handle_irrelevant(query)

In [66]:
# sample test
query = "Which medication is best for lowering LDL cholesterol in elderly patients?"
final_ans = agent_router(query)
print(final_ans)

 The choice of medication for lowering LDL cholesterol in elderly patients should be individualized based on factors such as the patient's overall health, other medications they are taking, and their specific cholesterol levels.

Statins are typically the first-line treatment for high LDL cholesterol. They are effective, well-tolerated, and have been shown to reduce the risk of cardiovascular disease. Examples of statins include atorvastatin (Lipitor), rosuvastatin (Crestor), and simvastatin (Zocor).

However, some elderly patients may not tolerate statins well due to side effects such as muscle pain or liver problems. In such cases, other medications may be considered.

Ezetimibe (Zetia) is a cholesterol-lowering medication that works by reducing the absorption of cholesterol from the gut. It can be used alone or in combination with a statin.

Fibrates, such as gemfibrozil (Lopid) and fenofibrate (Tricor), are another class of medications that can help lower LDL cholesterol. They work